Like exp2 but after removing some word from the assosiation that show problems and exluding the signer 9, i found it give some problem.

In [1]:
# imports
import numpy as np
import csv
import json

In [2]:
# File path

EXP_1_CSV_FILE_PATH = "/work/cvcs2024/SLR_sentiment_enhanced/SLRSE_model_data/Ensemble/SL-GCN/prediction/1/stat.csv"
EXP_2_CSV_FILE_PATH = "/work/cvcs2024/SLR_sentiment_enhanced/SLRSE_model_data/Ensemble/SL-GCN/prediction/2/stat.csv"
EXP_5_CSV_FILE_PATH = "/work/cvcs2024/SLR_sentiment_enhanced/SLRSE_model_data/Ensemble/SL-GCN/prediction/5/stat.csv"

EMOTION_LABEL_PATH = "/homes/mcolombari/SLR_Sentiment_Enhanced/DAN_module/sign_emotion_assosiation.json"

In [3]:
# Load emotion label

f_w_to_e = open(EMOTION_LABEL_PATH)
w_to_e = json.load(f_w_to_e)

raw_label = []
for _, i in w_to_e.items():
    for a in i:
        raw_label.append(a)

In [4]:
# Load data csv

exp_1_dict = dict()
with open(EXP_1_CSV_FILE_PATH, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        exp_1_dict[str(row[0])] = {
            'label': int(row[1]),
            'score_diff': float(row[2]),
            'score_diff_softmax': float(row[3]),
            'score': np.array([float(row[i + 4]) for i in range(2000)])
        }

exp_2_dict = dict()
with open(EXP_2_CSV_FILE_PATH, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        exp_2_dict[str(row[0])] = {
            'label': int(row[1]),
            'score_diff': float(row[2]),
            'score_diff_softmax': float(row[3]),
            'emotion': np.array([float(row[i + 4]) for i in range(8)]),
            'score': np.array([float(row[i + 12]) for i in range(2000)])
        }

exp_5_dict = dict()
with open(EXP_5_CSV_FILE_PATH, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        exp_5_dict[str(row[0])] = {
            'label': int(row[1]),
            'score_diff': float(row[2]),
            'score_diff_softmax': float(row[3]),
            'emotion': np.array([float(row[i + 4]) for i in range(8)]),
            'score': np.array([float(row[i + 12]) for i in range(2000)])
        }

## EXP 5

In [5]:
top1_count = sum([1 if el['label'] == el['score'].argsort()[-1] else 0 for _ , el in exp_5_dict.items()])
print(f"Top1 accuracy: {top1_count / len(exp_5_dict)}")

top5_count = sum([1 if el['label'] in el['score'].argsort()[-5:] else 0 for _ , el in exp_5_dict.items()])
print(f"Top5 accuracy: {top5_count / len(exp_5_dict)}")

top10_count = sum([1 if el['label'] in el['score'].argsort()[-10:] else 0 for _ , el in exp_5_dict.items()])
print(f"Top10 accuracy: {top10_count / len(exp_5_dict)}")

mean_loss_normal = np.array([el['score_diff'] for k, el in exp_5_dict.items()]).mean()
print(f"Mean loss normal: {mean_loss_normal}")
mean_loss_softmax = np.array([el['score_diff_softmax'] for k, el in exp_5_dict.items()]).mean()
print(f"Mean loss softmax: {mean_loss_softmax}")

Top1 accuracy: 0.30227565478746243
Top5 accuracy: 0.6277372262773723
Top10 accuracy: 0.7239158437097467
Mean loss normal: 726.2018297123229
Mean loss softmax: 1.461402962644912


Now it's finally better then exp1!!!

In [6]:
# Accuracy for emotion label

em_value = {}
for k, el in exp_5_dict.items():
    if el['label'] in raw_label:
        em_value[k] = el


top1_count = sum([1 if el['label'] == el['score'].argsort()[-1] else 0 for _ , el in em_value.items()])
print(f"Top1 accuracy: {top1_count / len(em_value)}")

top5_count = sum([1 if el['label'] in el['score'].argsort()[-5:] else 0 for _ , el in em_value.items()])
print(f"Top5 accuracy: {top5_count / len(em_value)}")

top10_count = sum([1 if el['label'] in el['score'].argsort()[-10:] else 0 for _ , el in em_value.items()])
print(f"Top10 accuracy: {top10_count / len(em_value)}")

mean_loss_normal = np.array([el['score_diff'] for k, el in em_value.items()]).mean()
print(f"Mean loss normal: {mean_loss_normal}")
mean_loss_softmax = np.array([el['score_diff_softmax'] for k, el in em_value.items()]).mean()
print(f"Mean loss softmax: {mean_loss_softmax}")

Top1 accuracy: 0.4358974358974359
Top5 accuracy: 0.7948717948717948
Top10 accuracy: 0.7948717948717948
Mean loss normal: 649.4470743589743
Mean loss softmax: 1.1504102564102563


Why do we get a worst result then exp2?

In [7]:
better_for_1 = ""
count_1 = 0
better_for_5 = ""
count_5 = 0

for name, el in exp_5_dict.items():
    pred_1 = exp_1_dict[name]['score'].argmax()
    pred_5 = exp_5_dict[name]['score'].argmax()
    label = exp_1_dict[name]['label']
    assert label == exp_5_dict[name]['label']
    if pred_1 == label and pred_5 != label:
        better_for_1 += f"""
            {name}\n
            {exp_1_dict[name]['score'].argsort()[-10:]}\n
            {exp_5_dict[name]['score'].argsort()[-10:]}\n
        """
        count_1 += 1
    if pred_5 == label and pred_1 != label:
        better_for_5 += f"""
            {name}\n
            {exp_1_dict[name]['score'].argsort()[-10:]}\n
            {exp_5_dict[name]['score'].argsort()[-10:]}\n
        """
        count_5 += 1

print(f"Better for 1\t{count_1}\n{better_for_1}")
print(f"Better for 5\t{count_5}\n{better_for_5}")

Better for 1	4

            signer13_sample67

            [1837 1001  317  243 1009  459 1343 1144  111  120]

            [1837 1001  243 1009  459 1343 1144  120  317  111]

        
            signer13_sample366

            [ 626 1528 1375 1350  895  530  817  539 1299  629]

            [ 626 1528 1375 1350  895  530  817  539  629 1299]

        
            signer32_sample160

            [  46  600 1268   49  796  234 1713  269 1373  846]

            [  46  600 1268   49  796  234 1713  269  846 1373]

        
            signer64_sample14

            [ 476  253  970  692 1111  206  831  468 1291  141]

            [ 476  253  970 1111  206  831  468  692  141 1291]

        
Better for 5	5

            signer5_sample582

            [ 605 1465  581 1244  591  351  321  179  309  313]

            [ 605 1465  581 1244  591  351  321  179  313  309]

        
            signer5_sample820

            [ 615  589  150    7 1244  125   98  237  492  775]

            [ 615  5

In [8]:
better_for_2 = ""
count_2 = 0
better_for_5 = ""
count_5 = 0

em_2_value = {}
for k, el in exp_2_dict.items():
    if el['label'] in raw_label:
        em_value[k] = el

em_5_value = {}
for k, el in exp_5_dict.items():
    if el['label'] in raw_label:
        em_value[k] = el

for name, el in em_5_value.items():
    pred_2 = exp_2_dict[name]['score'].argmax()
    pred_5 = exp_5_dict[name]['score'].argmax()
    label = exp_2_dict[name]['label']
    assert label == exp_5_dict[name]['label']
    if pred_2 == label and pred_5 != label:
        better_for_2 += f"""
            {name}\n
            {exp_2_dict[name]['score'].argsort()[-10:]}\n
            {exp_5_dict[name]['score'].argsort()[-10:]}\n
        """
        count_2 += 1
    if pred_5 == label and pred_2 != label:
        better_for_5 += f"""
            {name}\n
            {exp_2_dict[name]['score'].argsort()[-10:]}\n
            {exp_5_dict[name]['score'].argsort()[-10:]}\n
        """
        count_5 += 1

print(f"Better for 2\t{count_2}\n{better_for_2}")
print(f"Better for 5\t{count_5}\n{better_for_5}")

Better for 2	0

Better for 5	0



In [9]:
for k in exp_5_dict.keys():
    if 'signer_1' in k:
        print(k)